In [61]:
import pandas as pd
import numpy as np
import gensim 

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, log_loss
from sklearn import preprocessing
import re
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize

from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D


from tqdm import tqdm

nltk.download('stopwords')
nltk.download('punkt')
stops = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
train = pd.read_csv('train.csv')
print(len(train))
train.tail()

19579


,id,text,author
19574,id17718,"I could have fancied, while I looked at it, th...",EAP
19575,id08973,The lids clenched themselves together as if in...,EAP
19576,id05267,"Mais il faut agir that is to say, a Frenchman ...",EAP
19577,id17513,"For an item of news like this, it strikes us i...",EAP
19578,id00393,"He laid a gnarled claw on my shoulder, and it ...",HPL


In [11]:
test = pd.read_csv('test.csv')
print(len(test))
test.tail()

8392


,id,text
8387,id11749,All this is now the fitter for my purpose.
8388,id10526,I fixed myself on a wide solitude.
8389,id13477,It is easily understood that what might improv...
8390,id13761,"Be this as it may, I now began to feel the ins..."
8391,id04282,"Long winded, statistical, and drearily genealo..."


In [12]:
label_enconder = preprocessing.LabelEncoder()
label_enconder.fit(train['author'])
train['label_encoded'] = label_enconder.transform(train['author'])
train.head()

,id,text,author,label_encoded
0,id26305,"This process, however, afforded me no means of...",EAP,0
1,id17569,It never once occurred to me that the fumbling...,HPL,1
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP,0
3,id27763,How lovely is spring As we looked from Windsor...,MWS,2
4,id12958,"Finding nothing else, not even gold, the Super...",HPL,1


## Preprocessing function

In [13]:
def transformText(text):
    
    stops = set(stopwords.words("english"))
    
    # Convert text to lower
    text = text.lower()
    # Removing non ASCII chars    
    text = re.sub(r'[^\x00-\x7f]',r' ',text)
    
    # Strip multiple whitespaces
    text = gensim.corpora.textcorpus.strip_multiple_whitespaces(text)
    
    # Removing all the stopwords
    filtered_words = [word for word in text.split() if word not in stops]
    #filtered_words = [word for word in text.split()]

    # Removing all the tokens with lesser than 3 characters
    filtered_words = gensim.corpora.textcorpus.remove_short(filtered_words, minsize=3)
    
    # Preprocessed text after stop words removal
    text = " ".join(filtered_words)
    
    # Remove the punctuation
    text = gensim.parsing.preprocessing.strip_punctuation2(text)
    
    # Strip all the numerics
    text = gensim.parsing.preprocessing.strip_numeric(text)
    
    # Strip multiple whitespaces
    text = gensim.corpora.textcorpus.strip_multiple_whitespaces(text)
    
    # Stemming
    text = gensim.parsing.preprocessing.stem_text(text)
    return text

## Creating preprocessing column on train and testing

In [14]:
train['text_processed']=train['text'].apply(lambda x: transformText(x))
train.tail()

,id,text,author,label_encoded,text_processed
19574,id17718,"I could have fancied, while I looked at it, th...",EAP,0,could fanci look it emin landscap painter buil...
19575,id08973,The lids clenched themselves together as if in...,EAP,0,lid clench togeth spasm
19576,id05267,"Mais il faut agir that is to say, a Frenchman ...",EAP,0,mai faut agir sai frenchman never faint outright
19577,id17513,"For an item of news like this, it strikes us i...",EAP,0,item new like thi strike coolli receiv
19578,id00393,"He laid a gnarled claw on my shoulder, and it ...",HPL,1,laid gnarl claw shoulder seem shake altogeth m...


In [15]:
test['text_processed']=test['text'].apply(lambda x: transformText(x))
print(len(test))
test.tail()

8392


,id,text,text_processed
8387,id11749,All this is now the fitter for my purpose.,fitter purpos
8388,id10526,I fixed myself on a wide solitude.,fix wide solitud
8389,id13477,It is easily understood that what might improv...,easili understood might improv close scrutin d...
8390,id13761,"Be this as it may, I now began to feel the ins...",mai began feel inspir burn hope length nurtur ...
8391,id04282,"Long winded, statistical, and drearily genealo...",long wind statist drearili genealog matter wa ...


## Train/Test split

In [16]:
x_train, x_valid, y_train, y_valid = train_test_split(train['text_processed'], train['label_encoded'], test_size = 0.2, random_state = 4)
true_label = np.array(y_valid)
print("#" * 20 + " Some stats " + "#"*20)
print("Dataset training: {} uterances".format(x_train.shape[0]))
print("Dataset testing: {} uterances".format(x_valid.shape[0]))
print("Different classes: {}".format(len(y_train.unique())))

#################### Some stats ####################
Dataset training: 15663 uterances
Dataset testing: 3916 uterances
Different classes: 3


In [17]:
## Loading Glove vectors
embeddings_index = {}
f = open('../../vectors/glove.42B.300d.txt')
for line in tqdm(f):
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

1917495it [03:06, 10275.53it/s]

Found 1917495 word vectors.


In [19]:
word = 'awesome'
print(embeddings_index[word].shape)
embeddings_index[word][0:50]

(300,)


array([-0.080353  , -0.24218   , -0.26267001, -0.33482999,  0.27803999,
       -0.14549001, -2.7026999 , -0.75669998,  0.28336   , -0.72943997,
        0.21162   , -0.52950001,  0.31413001, -0.063769  , -0.1401    ,
       -0.15177999, -0.44237   , -0.48963001, -0.109     ,  0.20123   ,
        0.56939   , -0.13128   ,  0.37606001,  0.29006001,  0.10225   ,
        0.071932  ,  0.15706   ,  0.33153999, -0.012658  ,  0.73517001,
        0.30506   ,  0.16429999, -0.021664  ,  0.29159001, -0.10311   ,
       -0.46601   ,  0.10972   , -0.26945001,  0.37584001,  0.12056   ,
       -0.087698  , -0.051568  ,  0.20632   , -0.027614  ,  0.40970999,
        0.14663   ,  0.15049   , -0.44549   , -0.17702   ,  0.11271   ], dtype=float32)

In [36]:
# this function creates a normalized vector for the whole sentence
def sent2vec(s):
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if not w in stops]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(embeddings_index[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(300)
    return v / np.sqrt((v ** 2).sum())

In [37]:
## Create sentence vectors for the dataset
xtrain_glove = [sent2vec(x) for x in tqdm(x_train)]
xvalid_glove = [sent2vec(x) for x in tqdm(x_valid)]

100%|██████████| 3916/3916 [00:01<00:00, 3543.09it/s]


In [38]:
xtrain_glove = np.array(xtrain_glove)
xvalid_glove = np.array(xvalid_glove)

In [39]:
xtrain_glove.shape

(15663, 300)

In [40]:
xvalid_glove.shape

(3916, 300)

In [41]:
# scale the data before any neural net:
scl = preprocessing.StandardScaler()
xtrain_glove_scl = scl.fit_transform(xtrain_glove)
xvalid_glove_scl = scl.transform(xvalid_glove)

In [42]:
# we need to binarize the labels for the neural net
ytrain_enc = np_utils.to_categorical(y_train)
yvalid_enc = np_utils.to_categorical(y_valid)

## Building the model with simple 3 layer LSTM

In [46]:
model = Sequential()
model.add(Dense(300, input_dim=300, activation='relu'))
model.add(Dropout(0.8))
model.add(BatchNormalization())

model.add(Dense(300, activation='relu'))
model.add(Dropout(0.7))
model.add(BatchNormalization())

model.add(Dense(300, activation='relu'))
model.add(Dropout(0.7))
model.add(BatchNormalization())

model.add(Dense(3))
model.add(Activation('softmax'))

# compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [48]:
model.fit(xtrain_glove_scl, y=ytrain_enc, batch_size=64, 
          epochs=500, verbose=1, 
          validation_data=(xvalid_glove_scl, yvalid_enc))

Train on 15663 samples, validate on 3916 samples
Epoch 1/500
15663/15663 [==============================] - 5s 308us/step - loss: 0.6519 - val_loss: 0.7326
Epoch 2/500
15663/15663 [==============================] - 5s 308us/step - loss: 0.6541 - val_loss: 0.7350
Epoch 3/500
15663/15663 [==============================] - 5s 310us/step - loss: 0.6569 - val_loss: 0.7339
Epoch 4/500
15663/15663 [==============================] - 5s 318us/step - loss: 0.6505 - val_loss: 0.7337
Epoch 5/500
15663/15663 [==============================] - 5s 316us/step - loss: 0.6548 - val_loss: 0.7339
Epoch 6/500
15663/15663 [==============================] - 5s 320us/step - loss: 0.6517 - val_loss: 0.7341
Epoch 7/500
15663/15663 [==============================] - 5s 316us/step - loss: 0.6486 - val_loss: 0.7374
Epoch 8/500
15663/15663 [==============================] - 5s 315us/step - loss: 0.6548 - val_loss: 0.7350
Epoch 9/500
15663/15663 [==============================] - 5s 317us/step - loss: 0.6446 - val_l

15663/15663 [==============================] - 5s 313us/step - loss: 0.6029 - val_loss: 0.7323
Epoch 77/500
15663/15663 [==============================] - 5s 317us/step - loss: 0.6133 - val_loss: 0.7300
Epoch 78/500
15663/15663 [==============================] - 5s 315us/step - loss: 0.6046 - val_loss: 0.7330
Epoch 79/500
15663/15663 [==============================] - 5s 317us/step - loss: 0.5991 - val_loss: 0.7395
Epoch 80/500
15663/15663 [==============================] - 5s 314us/step - loss: 0.6124 - val_loss: 0.7330
Epoch 81/500
15663/15663 [==============================] - 5s 315us/step - loss: 0.6031 - val_loss: 0.7279
Epoch 82/500
15663/15663 [==============================] - 5s 318us/step - loss: 0.6041 - val_loss: 0.7274
Epoch 83/500
15663/15663 [==============================] - 5s 321us/step - loss: 0.5981 - val_loss: 0.7315
Epoch 84/500
15663/15663 [==============================] - 5s 322us/step - loss: 0.6006 - val_loss: 0.7288
Epoch 85/500
15663/15663 [===============

15663/15663 [==============================] - 5s 318us/step - loss: 0.5758 - val_loss: 0.7269
Epoch 152/500
15663/15663 [==============================] - 5s 314us/step - loss: 0.5762 - val_loss: 0.7256
Epoch 153/500
15663/15663 [==============================] - 5s 313us/step - loss: 0.5807 - val_loss: 0.7290
Epoch 154/500
15663/15663 [==============================] - 5s 312us/step - loss: 0.5704 - val_loss: 0.7218
Epoch 155/500
15663/15663 [==============================] - 5s 324us/step - loss: 0.5753 - val_loss: 0.7215
Epoch 156/500
15663/15663 [==============================] - 5s 324us/step - loss: 0.5729 - val_loss: 0.7195
Epoch 157/500
15663/15663 [==============================] - 5s 319us/step - loss: 0.5743 - val_loss: 0.7251
Epoch 158/500
15663/15663 [==============================] - 5s 317us/step - loss: 0.5717 - val_loss: 0.7226
Epoch 159/500
15663/15663 [==============================] - 5s 315us/step - loss: 0.5659 - val_loss: 0.7225
Epoch 160/500
15663/15663 [======

15663/15663 [==============================] - 5s 313us/step - loss: 0.5560 - val_loss: 0.7241
Epoch 227/500
15663/15663 [==============================] - 5s 319us/step - loss: 0.5437 - val_loss: 0.7342
Epoch 228/500
15663/15663 [==============================] - 5s 322us/step - loss: 0.5555 - val_loss: 0.7206
Epoch 229/500
15663/15663 [==============================] - 5s 318us/step - loss: 0.5639 - val_loss: 0.7230
Epoch 230/500
15663/15663 [==============================] - 5s 310us/step - loss: 0.5455 - val_loss: 0.7222
Epoch 231/500
15663/15663 [==============================] - 5s 313us/step - loss: 0.5513 - val_loss: 0.7306
Epoch 232/500
15663/15663 [==============================] - 5s 311us/step - loss: 0.5562 - val_loss: 0.7373
Epoch 233/500
15663/15663 [==============================] - 5s 322us/step - loss: 0.5526 - val_loss: 0.7197
Epoch 234/500
15663/15663 [==============================] - 5s 321us/step - loss: 0.5500 - val_loss: 0.7247
Epoch 235/500
15663/15663 [======

15663/15663 [==============================] - 5s 313us/step - loss: 0.5328 - val_loss: 0.7263
Epoch 302/500
15663/15663 [==============================] - 5s 317us/step - loss: 0.5300 - val_loss: 0.7262
Epoch 303/500
15663/15663 [==============================] - 5s 321us/step - loss: 0.5403 - val_loss: 0.7293
Epoch 304/500
15663/15663 [==============================] - 5s 309us/step - loss: 0.5378 - val_loss: 0.7360
Epoch 305/500
15663/15663 [==============================] - 5s 310us/step - loss: 0.5290 - val_loss: 0.7303
Epoch 306/500
15663/15663 [==============================] - 5s 314us/step - loss: 0.5329 - val_loss: 0.7245
Epoch 307/500
15663/15663 [==============================] - 5s 322us/step - loss: 0.5350 - val_loss: 0.7238
Epoch 308/500
15663/15663 [==============================] - 5s 316us/step - loss: 0.5321 - val_loss: 0.7241
Epoch 309/500
15663/15663 [==============================] - 5s 315us/step - loss: 0.5346 - val_loss: 0.7243
Epoch 310/500
15663/15663 [======

15663/15663 [==============================] - 5s 320us/step - loss: 0.5271 - val_loss: 0.7326
Epoch 377/500
15663/15663 [==============================] - 5s 315us/step - loss: 0.5138 - val_loss: 0.7296
Epoch 378/500
15663/15663 [==============================] - 5s 316us/step - loss: 0.5263 - val_loss: 0.7270
Epoch 379/500
15663/15663 [==============================] - 5s 321us/step - loss: 0.5121 - val_loss: 0.7245
Epoch 380/500
15663/15663 [==============================] - 5s 322us/step - loss: 0.5269 - val_loss: 0.7339
Epoch 381/500
15663/15663 [==============================] - 5s 322us/step - loss: 0.5187 - val_loss: 0.7357
Epoch 382/500
15663/15663 [==============================] - 5s 315us/step - loss: 0.5158 - val_loss: 0.7279
Epoch 383/500
15663/15663 [==============================] - 5s 317us/step - loss: 0.5273 - val_loss: 0.7331
Epoch 384/500
15663/15663 [==============================] - 5s 316us/step - loss: 0.5107 - val_loss: 0.7355
Epoch 385/500
15663/15663 [======

10432/15663 [==================>...........] - ETA: 1s - loss: 0.4976

KeyboardInterrupt: 

In [49]:
preds_proba = model.predict_proba(xvalid_glove)
print("Log-loss: {0:.3f}".format(log_loss(true_label, preds_proba)))

Log-loss: 1.710


In [54]:
preds_proba.shape

(3916, 3)

## Tokenize text data

In [57]:
token = text.Tokenizer(num_words=None)
max_len = 70

token.fit_on_texts(list(x_train) + list(x_valid))
xtrain_seq = token.texts_to_sequences(x_train)
xvalid_seq = token.texts_to_sequences(x_valid)

# zero pad the sequences
xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen=max_len)

word_index = token.word_index

In [59]:
# create an embedding matrix for the words we have in the dataset
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

100%|██████████| 15565/15565 [00:00<00:00, 208710.20it/s]


In [62]:
# A simple LSTM with glove embeddings and two dense layers
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [63]:
model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100, verbose=1, validation_data=(xvalid_pad, yvalid_enc))

Train on 15663 samples, validate on 3916 samples
Epoch 1/100
15663/15663 [==============================] - 83s 5ms/step - loss: 1.0904 - val_loss: 1.0416
Epoch 2/100
15663/15663 [==============================] - 82s 5ms/step - loss: 1.0099 - val_loss: 0.8789
Epoch 3/100
15663/15663 [==============================] - 81s 5ms/step - loss: 0.9260 - val_loss: 0.8298
Epoch 4/100
15663/15663 [==============================] - 82s 5ms/step - loss: 0.8989 - val_loss: 0.8057
Epoch 5/100
15663/15663 [==============================] - 82s 5ms/step - loss: 0.8859 - val_loss: 0.7928
Epoch 6/100
15663/15663 [==============================] - 82s 5ms/step - loss: 0.8644 - val_loss: 0.7759
Epoch 7/100
15663/15663 [==============================] - 81s 5ms/step - loss: 0.8513 - val_loss: 0.7564
Epoch 8/100
15663/15663 [==============================] - 82s 5ms/step - loss: 0.8331 - val_loss: 0.7417
Epoch 9/100
15663/15663 [==============================] - 82s 5ms/step - loss: 0.8246 - val_loss: 0.75

KeyboardInterrupt: 

In [65]:
preds_proba = model.predict_proba(xvalid_pad)
print("Log-loss: {0:.3f}".format(log_loss(true_label, preds_proba)))

Log-loss: 0.722


## Generating submissions

In [66]:
my_sub = pd.DataFrame(columns={'id', 'EAP','HPL', 'MWS'})
my_sub=my_sub[['id', 'EAP','HPL', 'MWS']]
my_sub

,id,EAP,HPL,MWS


In [67]:
## Finish this
for i in range(len(test)):
    query = test['text_processed'][i]
    query = cnt.transform([query])
    preds_proba = model.predict_proba(query.tocsc())
    print(i)
    if not preds_proba.size:
        print("não tem reposta")
        preds_proba=np.array([[0.4, 0.3, 0.3]])
    my_sub.loc[i] = [test['id'][i], preds_proba[0][0], preds_proba[0][1], preds_proba[0][2]]

NameError: name 'cnt' is not defined